In [11]:
import numpy as np
import torch
import time
import jax.numpy as jnp

In [12]:
def bench(name, func, warmup=True):
    """Benchmark a function with optional warmup"""
    if warmup:
        func()  # Warmup run
    start = time.time()
    result = func()
    end = time.time()
    elapsed = end - start
    print(f"{name:40s}: {elapsed:8.6f} sec")
    return elapsed

In [17]:
import pandas as pd

# Test all three required matrix sizes
sizes = [100, 1000, 10000]
results = []

for size in sizes:
    print(f"\n{'='*60}")
    print(f"Matrix Size: {size}×{size}")
    print(f"{'='*60}")
    
    # Generate matrices
    np.random.seed(42)
    A_np = np.random.rand(size, size).astype(np.float32)
    B_np = np.random.rand(size, size).astype(np.float32)
    A_torch = torch.tensor(A_np)
    B_torch = torch.tensor(B_np)
    
    # Benchmark each library
    numpy_time = bench("NumPy (@)", lambda: A_np @ B_np)
    pytorch_time = bench("PyTorch CPU", lambda: A_torch @ B_torch)
    jax_time = bench("JAX (XLA)", lambda: jnp.dot(A_np, B_np).block_until_ready())
    
    results.append({
        'Size': f'{size}×{size}',
        'NumPy (s)': numpy_time,
        'PyTorch (s)': pytorch_time,
        'JAX (s)': jax_time,
        'Best': min(numpy_time, pytorch_time, jax_time)
    })

print(f"\n{'='*60}")
print("SUMMARY TABLE")
print(f"{'='*60}")

# Create comparison table
df = pd.DataFrame(results)
df['Best Library'] = df[['NumPy (s)', 'PyTorch (s)', 'JAX (s)']].idxmin(axis=1).str.replace(' (s)', '')
print(df.to_string(index=False))
print()


Matrix Size: 100×100
NumPy (@)                               : 0.000021 sec
PyTorch CPU                             : 0.000020 sec
JAX (XLA)                               : 0.000041 sec

Matrix Size: 1000×1000
NumPy (@)                               : 0.004713 sec
PyTorch CPU                             : 0.011616 sec
JAX (XLA)                               : 0.003049 sec

Matrix Size: 10000×10000
NumPy (@)                               : 2.147201 sec
NumPy (@)                               : 2.147201 sec
PyTorch CPU                             : 2.200503 sec
PyTorch CPU                             : 2.200503 sec
JAX (XLA)                               : 1.709231 sec

SUMMARY TABLE
       Size  NumPy (s)  PyTorch (s)  JAX (s)     Best Best Library
    100×100   0.000021     0.000020 0.000041 0.000020      PyTorch
  1000×1000   0.004713     0.011616 0.003049 0.003049          JAX
10000×10000   2.147201     2.200503 1.709231 1.709231          JAX

JAX (XLA)                              

In [18]:
import sys
import os

print("="*60)
print("SYSTEM INFORMATION")
print("="*60)
print(f"\nPython: {sys.version.split()[0]}")
print(f"NumPy: {np.__version__}")
print(f"PyTorch: {torch.__version__}")
print(f"JAX: {jnp.__version__ if hasattr(jnp, '__version__') else 'N/A'}")

print(f"\nCPU Cores: {os.cpu_count()}")
print(f"PyTorch Threads: {torch.get_num_threads()}")

# Check BLAS backend
print("\nBLAS Backend:")
if hasattr(np, 'show_config'):
    np.show_config()
else:
    print("  Unable to determine NumPy BLAS backend")
    
print("="*60)

SYSTEM INFORMATION

Python: 3.11.13
NumPy: 2.3.3
PyTorch: 2.9.0+cu130
JAX: N/A

CPU Cores: 20
PyTorch Threads: 14

BLAS Backend:
Build Dependencies:
  blas:
    detection method: pkgconfig
    found: true
    include directory: /opt/_internal/cpython-3.11.13/lib/python3.11/site-packages/scipy_openblas64/include
    lib directory: /opt/_internal/cpython-3.11.13/lib/python3.11/site-packages/scipy_openblas64/lib
    name: scipy-openblas
    openblas configuration: OpenBLAS 0.3.30  USE64BITINT DYNAMIC_ARCH NO_AFFINITY
      Haswell MAX_THREADS=64
    pc file directory: /project/.openblas
    version: 0.3.30
  lapack:
    detection method: pkgconfig
    found: true
    include directory: /opt/_internal/cpython-3.11.13/lib/python3.11/site-packages/scipy_openblas64/include
    lib directory: /opt/_internal/cpython-3.11.13/lib/python3.11/site-packages/scipy_openblas64/lib
    name: scipy-openblas
    openblas configuration: OpenBLAS 0.3.30  USE64BITINT DYNAMIC_ARCH NO_AFFINITY
      Haswell MA

## Analysis: Why Libraries Are Faster

Production libraries achieve superior performance through:
1. **SIMD Vectorization**: AVX2/AVX-512 instructions process 8-16 floats per instruction
2. **Optimized BLAS**: Hand-tuned assembly kernels (Intel MKL, OpenBLAS)
3. **Cache Blocking**: Adaptive tile sizes based on CPU cache hierarchy
4. **Prefetching**: Hardware prefetch hints to reduce memory latency
5. **Multi-threading**: Automatic parallelization across all cores

Our educational implementations demonstrate parallel computing concepts without these low-level optimizations.

In [21]:
# Our implementation results (from actual test runs on same hardware - Machine 3)
our_results = {
    '100×100': {
        'OpenMP Naive (1 thread)': 0.0070,
        'OpenMP Naive (16 threads)': 0.0018,
        'OpenMP Strassen (7 threads)': 0.0031
    },
    '1000×1000': {
        'OpenMP Naive (1 thread)': 0.1897,
        'OpenMP Naive (16 threads)': 0.1009,
        'OpenMP Strassen (7 threads)': 0.1800
    },
    '10000×10000': {
        'OpenMP Naive (1 thread)': 158.79,
        'OpenMP Naive (16 threads)': 57.55,
        'OpenMP Strassen (7 threads)': 118.78
    }
}

print("\n" + "="*80)
print("COMPARISON: Optimized Libraries vs Our C++/OpenMP Implementations")
print("="*80)

for i, size in enumerate(sizes):
    size_str = f'{size}×{size}'
    
    # Get library results
    numpy_time = results[i]['NumPy (s)']
    pytorch_time = results[i]['PyTorch (s)']
    jax_time = results[i]['JAX (s)']
    lib_best = results[i]['Best']
    
    # Determine which library is best
    if lib_best == numpy_time:
        lib_best_name = 'NumPy'
    elif lib_best == pytorch_time:
        lib_best_name = 'PyTorch'
    else:
        lib_best_name = 'JAX'
    
    # Get our best result
    our_best = min(our_results[size_str].values())
    our_best_name = min(our_results[size_str].items(), key=lambda x: x[1])[0]
    
    print(f"\n{size_str}:")
    print(f"  Best Library ({lib_best_name}):     {lib_best:10.6f}s")
    print(f"  Our Best ({our_best_name}): {our_best:10.6f}s")
    print(f"  Performance Gap (Our/Library):      {our_best/lib_best:10.2f}×")
    
print("\n" + "="*80)



COMPARISON: Optimized Libraries vs Our C++/OpenMP Implementations

100×100:
  Best Library (PyTorch):       0.000020s
  Our Best (OpenMP Naive (16 threads)):   0.001800s
  Performance Gap (Our/Library):           88.82×

1000×1000:
  Best Library (JAX):       0.003049s
  Our Best (OpenMP Naive (16 threads)):   0.100900s
  Performance Gap (Our/Library):           33.09×

10000×10000:
  Best Library (JAX):       1.709231s
  Our Best (OpenMP Naive (16 threads)):  57.550000s
  Performance Gap (Our/Library):           33.67×



## Comprehensive Benchmark: All Matrix Sizes

# Library Comparison Benchmarks

This notebook compares our implementations against production-grade matrix multiplication libraries:
- **NumPy**: Standard Python numerical library (uses BLAS/LAPACK)
- **PyTorch**: Deep learning framework (uses MKL or OpenBLAS)
- **JAX**: Google's high-performance numerical computing library (XLA compiler)

We test three matrix sizes: 100×100, 1000×1000, and 10000×10000 to match our C++/OpenMP/MPI implementations.